In [ ]:
# Run from the root of the repo
%cd ..

In [ ]:
import duckdb

# Connect to a DuckDB database (in-memory for this example, or specify a file path)
con = duckdb.connect(database=":memory:", read_only=False)

# Install the Avro extension
con.execute("INSTALL avro;")
con.execute("LOAD avro;")

# Install the Spatial extension
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Install the JSON extension
con.execute("INSTALL json; LOAD json;")

In [ ]:
# Run a select statement on the Avro files, taking the object_id, ra, and
# dec columns, and filtering for a specific object_id
# ra and dec are inside a candidate object, so we need to extract them

con.execute(
    """
        CREATE OR REPLACE TABLE alert AS
        SELECT
            objectId AS object_id,
            candidate.ra AS ra,
            candidate.dec AS dec
        FROM read_avro('data/ztf_public_20250614/*.avro');
    """
)

In [ ]:
con.execute(
    """
        select count(*) from alert
    """
).fetchall()

In [ ]:
con.execute("select distinct object_id from alert").fetchall()

In [ ]:
from math import pi
import time

# Postgres query for reference
sql = """
    SELECT *
    FROM alert
    WHERE q3c_radial_query(ra, dec, 180.0, 0.0, 1.0)
    LIMIT 20;
"""

# Define cone search parameters
search_ra = 180.0  # Example: Right Ascension of cone center in degrees
search_dec = 0.0   # Example: Declination of cone center in degrees
cone_radius_arcsec = 3600 # Example: 300 arcseconds (5 arcminutes)
cone_radius_deg = cone_radius_arcsec / 3600.0 # Convert arcseconds to degrees

# SQL query to perform the cone search using spherical law of cosines
# (Often good enough for small radii, more complex for robustness or larger radii)
# For very small radii, you might use a simplified planar approximation or more robust spherical formulas.
# Let's use the spherical law of cosines for a common approach.

# Convert degrees to radians for trigonometric functions
search_ra_rad = search_ra * pi / 180.0
search_dec_rad = search_dec * pi / 180.0

query = f"""
    SELECT
        *,
        ACOS(
            SIN(RADIANS(dec)) * SIN({search_dec_rad}) +
            COS(RADIANS(dec)) * COS({search_dec_rad}) *
            COS(ABS(RADIANS(ra) - {search_ra_rad}))
        ) * 180.0 / PI() AS angular_distance_deg
    FROM
        alert
    WHERE
        angular_distance_deg <= {cone_radius_deg};
"""

# Execute the query
t0 = time.time()
result = con.execute(query).fetchall()
print("Time (s)", time.time() - t0)

print(f"Found {len(result)} objects within the cone.")
print(result)